In [ ]:
import sys, os
import time
import cv2

import datetime                                                                 
import pickle, bz2 

import tensorflow as tf
import numpy as np

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../../utils'))

In [ ]:
RLTrajFile = "./RLTrajRec_2020-08-31_00-15-00"

infile = bz2.BZ2File(RLTrajFile, 'r')
RLTrajDict = pickle.load(infile)
infile.close()

In [ ]:
print(RLTrajDict.keys())

In [ ]:
print("Ep. length =",RLTrajDict["epLen"] )

In [ ]:
for key, value in RLTrajDict.items():
    if type(value) == list:
        print("len(",key,") :", len(value))
    else:
        print(key,":", value)

In [ ]:
additionalPar = int(RLTrajDict["addObs"][0][0,0])

# 1P
nScalarAddPar = additionalPar - 2*RLTrajDict["nChars"]\
                - RLTrajDict["actBufLen"]*(RLTrajDict["nActions"][0]+RLTrajDict["nActions"][1])

print("Additional Par = ", additionalPar)
print("N scalar actions = ", nScalarAddPar)

cumulativeEpRew = 0.0
cumulativeEpRewAll = []

for idx in range(RLTrajDict["epLen"]):
    
    # 1P
    action = RLTrajDict["actions"][idx]

    print("Action:", action)
    
    obsH = RLTrajDict["frames"][0].shape[0]
    obsW = RLTrajDict["frames"][0].shape[1]
    obsNChannels = len(RLTrajDict["frames"]) - RLTrajDict["epLen"] + 1
    observation = np.zeros((obsH, obsW, obsNChannels))
    
    for iFrame in range(obsNChannels-1):
        observation[:,:,iFrame] = RLTrajDict["frames"][idx + 1 + iFrame]
    observation[:,:,obsNChannels-1] = RLTrajDict["addObs"][idx]
        
    reward = RLTrajDict["rewards"][idx]
    
    done = False
    if idx == RLTrajDict["epLen"] - 1:
        done = True

    addPar = RLTrajDict["addObs"][idx]
    addPar = np.reshape(addPar, (-1))
    addPar = addPar[1:additionalPar+1]
    actions = addPar[0:additionalPar-nScalarAddPar-2*RLTrajDict["nChars"]]
        
    limAct = [RLTrajDict["actBufLen"] * RLTrajDict["nActions"][0], 
              RLTrajDict["actBufLen"] * RLTrajDict["nActions"][0] + 
              RLTrajDict["actBufLen"] * RLTrajDict["nActions"][1]]
        
    moveActionsP1   = actions[0:limAct[0]]
    attackActionsP1 = actions[limAct[0]:limAct[1]]
    moveActionsP1   = np.reshape(moveActionsP1, (RLTrajDict["actBufLen"],-1))
    attackActionsP1 = np.reshape(attackActionsP1, (RLTrajDict["actBufLen"],-1))
    print("Move actions P1 = ", moveActionsP1)
    print("Attack actions P1 = ", attackActionsP1)
        
    others = addPar[additionalPar-nScalarAddPar-2*RLTrajDict["nChars"]:]
    print("ownHealth = ", others[0])
    print("oppHealth = ", others[1])
    print("ownPosition = ", others[2])
    print("oppPosition = ", others[3])
    #print("stage = ", others[4])
    #input("Pausa1")       
       
    obs = np.array(observation).astype(np.float32)
    
    for iFrame in range(obsNChannels-1):
        cv2.imshow("image"+str(iFrame), obs[:,:,iFrame])
    
    cv2.waitKey()
    
    cumulativeEpRew += reward
    
    if np.any(done):
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        sys.stdout.flush()
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0
        input("Stop")    